# McWhorter Problem
<cite>[McWhorter and Sunada][1]</cite> propose an analytical solution to the two-phase flow equation. A one-dimensional problem was considered which describes the flow of two incompressible, immiscible fluids through a porous medium, where the wetting phase (water) displaces the non-wetting fluid (air or oil) in the horizontal direction (without the influence of gravity).

![mcWhorter_concept_schematic.png](figures/mcWhorter_concept.png)

[1]: https://doi.org/10.1029/WR026i003p00399

### Material and Problem parameter

The parameter for this problem are stored in a CSV file. Let's look at the table below. The column "Key" shows the parameter name in the PRJ file. The column "Value" shows current parameter value. The columns "Range min", "Range max" and "Scale" are used in the next section to define sliders to interact with the analytic solution.

In [33]:
from helper import render_latex_table
import pandas as pd

df = pd.read_csv("parameter.csv", quotechar="'")

# We look at this column later
df_OGSTools = df[["Key", "OGSTools"]]
df.drop('OGSTools', axis=1, inplace=True)

render_latex_table(df, latex_column="Symbol")

Property,Symbol,Key,Value,Range min,Range max,Scale,Unit
Porosity,$$ \varphi $$,phi,0.3,0.05,0.3,linear,1
Intrinsic permeability,$$ K $$,K,1.0000e-10,1.0000e-12,1.0000e-08,log,m²
Residual saturation of the wetting phase,$$ S_L^{res} $$,Swr,0.02,0.0,0.3,linear,1
Residual saturation of the non-wetting phase,$$ S_G^{res} $$,Snr,0.001,0.0,0.3,linear,1
Dynamic viscosity of the wetting phase,$$ \mu_L $$,mun,0.001,0.0001,0.01,log,Pa·s
Dynamic viscosity of the non-wetting pha,$$ \mu_G $$,muw,0.001,0.0001,0.01,log,Pa·s
Brooks and Corey model parameter: entry pressure,$$ p_b $$,Pe,5000,1000,100000,log,Pa
Brooks and Corey model parameter: pore size distribution index,$$ \lambda $$,lambda_,3,1.5,3,linear,1
Initial saturation,$$ S_L(t=0) $$,Si,0.2,0.0,0.5,linear,1
Injection boundary saturation,$$ S_L(x=0) $$,S0,0.9,0.51,1,linear,1


<IPython.core.display.Javascript object>

## Analytical solution
The analytical solution is calculated according to <cite>[McWhorter and Sunada][1]</cite>. The intial saturation is Si in the domain. At $t=0$ the saturation at $x=0$ becomes S0. The analytical solution is calculated for $t=1000$ s.

[1]: https://doi.org/10.1029/WR026i003p00399

In [34]:
from mcworther import BrooksCorey, McWorther
import numpy as np
import ipywidgets as widgets
from IPython.display import display

def plot_f(mun, muw, K, phi, lambda_, Pe, S0, Si, **kwargs):
    model = BrooksCorey(Pe, lambda_)
    problem = McWorther(model, phi, K, muw, mun, S0, Si)
    problem.plot_solution()

# Erstellen von Schiebereglern für interaktive Plots basierend auf dem DataFrame
sliders = {}
for idx, row in df.iterrows():
    key = row['Key']
    if row['Scale'] == "log":
        sliders[key] = widgets.FloatLogSlider(
            value=row['Value'], base=10, min=np.log10(row['Range min']),
            max=np.log10(row['Range max']), step=0.01, description=key
        )
    else:
        sliders[key] = widgets.FloatSlider(
            value=row['Value'], min=row['Range min'], max=row['Range max'],
            step=0.01, description=key
        )

# Darstellung der Schieberegler und des Plots
ui = widgets.VBox(list(sliders.values()))
out = widgets.interactive_output(plot_f, sliders)
display(out)
display(ui)


Output()

## Numerical solutions

For the numerical solution, we compare the Thermal-2-Phase-Hydro-Mechanical (TH2M) and the Two-phase Flow formulation.

1. The first step is to create a matching mesh that capture the penetration depth of the wetting fluid. 
2. Because the OGS models use the capillary pressure as a primary variable, the boundary and initial conditions need to be converted from saturations to pressures.

Tip: Pick the parameters you would like to study with the sliders above and run the code blocks below to compare the results of the analytical and numerical model.

In [ ]:
import pyvista as pv
from helper import create_1d_mesh, create_boundary_line_meshes
from template import prj_from_template

#parameters defined by the current slider settings
params = {k: v.value for k, v in sliders.items()}

#calculate analytical solution with the 
model = BrooksCorey(params["Pe"], params["lambda_"])
problem = McWorther(model, params["phi"], params["K"], params["muw"], params["mun"], params["S0"], params["Si"])
x_ref, Sw_ref = problem.get_solution()

# 1. Create meshes according to the penetration depth of the wetting fluid

depth = max(x_ref)      # penetration depth of the wetting fluid
factor = 5.0   # mesh should be factor times larger than the depth 
Nel = 100         # number of mesh elements

print(f"penetration depth of the wetting fluid: {depth}")

# 1.1 Mesh for TwoPhaseFlowPP
mesh = create_1d_mesh(point_a=(0.0, 0.0, 0.0), point_b=(depth*factor, 0.0, 0.0), num_points=Nel+1, mesh_type="line")
mesh.save(r"mesh_line.vtu")

coords = {'x1': mesh.GetBounds()[0], 'x2': 0.5*mesh.GetBounds()[0]}
prj_from_template(coords, "mcwt.template", "mcwt.gml")

# 1.2 Mesh for TH2M
mesh = create_1d_mesh(point_a=(0.0, 0.0, 0.0), point_b=(depth*factor, 0.0, 0.0), num_points=Nel+1, mesh_type="quad")
mesh.save(r"mesh_quad.vtu")

boundary_left, boundary_right = create_boundary_line_meshes(point_a=(0.0, 0.0, 0.0), point_b=(depth*factor, 0.0, 0.0), num_points=Nel+1)
boundary_left.save("boundary_left_test.vtu")                                   

#!checkMesh -v mesh_quad.vtu
#!identifySubdomains -m mesh_quad.vtu -- boundary_left_test.vtu
#!checkMesh -v boundary_left_test.vtu


# 2. Calculate capillary pressures  
params["pci"] = model.pc(params["Si"])
params["pc0"] = model.pc(params["S0"])

print(f"Si = {params["Si"]} -> pci = {params["pci"]}")
print(f"S0 = {params["S0"]} -> pc0 = {params["pc0"]}")

penetration depth of the wetting fluid: 0.7370002578514665
2 replacements made.
8549.879733383485
5178.720843256431


Now we can update the PRJ files and run the simulations.

In [ ]:
import json

replace_info = dict()

for index, row in df_OGSTools.iterrows():
    if row["OGSTools"] == "Parameter":
        replace_info[row["Key"]] = row["OGSTools"]
    else:
        replace_info[row["Key"]] = json.loads(row["OGSTools"])

# Add infos for calculated parameter
replace_info["pci"] = "Parameter"
replace_info["pc0"] = "Parameter"

In [37]:
import ogstools as ogs
from template import prj_from_template

# Simulate problem with TwoPhaseFlowPP and TH2M
prj_files = [r"mcWhorter_TwoPhasePP.prj", r"mcWhorter_TH2M.prj"]

for prj_file in prj_files:
    prj = ogs.Project(input_file=prj_file, output_file=prj_file)
    for key in params.keys():
        try:
            if replace_info[key] == "Parameter":
                prj.replace_parameter_value(name=key, value=params[key])
            else:
                for r in replace_info[key]:
                    prj.replace_medium_property_value(mediumid=r["medium id"],
                                                      name=r["name"],
                                                      value=params[key],
                                                      valuetag=r["valuetag"])
        except:
            print(f"Parameter {key} not found...")
    prj.write_input()        
    prj.run_model()

ModuleNotFoundError: No module named 'ogstools'

## Evaluation and Results

In [ ]:
import numpy as np
import pyvista as pv
from helper import plot_with_error

# 1. Plot analytical solution
plotter = plot_with_error(x_ref, Sw_ref, "C0-",
                          "Analytical solution",
                          "Distance $x$ (m)",
                          "Wetting saturation $S_w$ (-)",
                          "Error (Numerical - Analytical) (-)")

# 2. Read results from TwoPhaseFlowPP
labels = ["TwoPhaseFlowPP", "TH2M"]
results = ["twophaseflow_test_t_1000.000000.vtu", "result_McWhorter_H2_ts_110_t_1000.000000.vtu"]

i = 1
for label, result in zip(labels, results):

    mesh = pv.read(result)
    print(mesh.point_data.keys())

    Sw=mesh["saturation"]
    x=mesh.points[:,0]

    ind = np.argsort(x)
    x = x[ind]
    Sw = Sw[ind]

    plotter.append(x, Sw, f"C{i}x", label)
    i = i + 1 

# show plot

plotter.plot([0, depth*1.2])

## Literature

McWhorter, D. B., and D. K. Sunada (1990), Exact integral solutions for two-phase flow, Water Resour. Res., 26(3), 399–413, <cite>[doi:10.1029/WR026i003p00399][1]</cite>. 

[1]: https://doi.org/10.1029/WR026i003p00399
